## 1. Basic import

In [1]:
from sklearn import svm
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPClassifier

### Real time output in notebook to follow training

In [86]:
import sys
oldsysstdout = sys.stdout
class flushfile():
    def __init__(self, f):
        self.f = f
    def __getattr__(self,name): 
        return object.__getattribute__(self.f, name)
    def write(self, x):
        self.f.write(x)
        self.f.flush()
    def flush(self):
        self.f.flush()
sys.stdout = flushfile(sys.stdout)

## 2. Creation of dataset : features and labels 

In [91]:
XY = pd.read_pickle('MLbodyFace').dropna()

In [92]:
XY.columns

Index([u'seq', u'id', u'traSeq', u'trackId', u'trackX', u'trackY', u'trackW',
       u'trackH', u'height', u'distance', u'rgbSeq', u'blur', u'embedding',
       u'GroundTrue'],
      dtype='object')

In [93]:
XY.drop(['seq', 'id', 'traSeq', 'trackId', 'rgbSeq'], axis=1, inplace=True)

In [94]:
XY

,trackX,trackY,trackW,trackH,height,distance,blur,embedding,GroundTrue
0,326.4,124.0,387.6,458.0,1.571027,2.248792,367.316834,"[-0.110296, 0.0504573, 0.00468473, -0.0105098,...",Oliver
1,802.4,150.0,442.0,520.0,1.519628,1.882822,790.153174,"[0.0554316, 0.0860607, -0.00389893, 0.0320879,...",Alberto
2,1135.6,108.0,309.4,364.0,1.475796,2.760290,600.989129,"[-0.0656685, 0.047579, -0.100778, 0.00661445, ...",Lucas
4,323.0,124.0,387.6,456.0,1.569231,2.253340,367.316834,"[-0.110296, 0.0504573, 0.00468473, -0.0105098,...",Oliver
5,805.8,152.0,438.6,518.0,1.519507,1.894786,790.153174,"[0.0554316, 0.0860607, -0.00389893, 0.0320879,...",Alberto
6,1128.8,104.0,312.8,370.0,1.486020,2.723061,600.989129,"[-0.0656685, 0.047579, -0.100778, 0.00661445, ...",Lucas
12,306.0,124.0,387.6,458.0,1.567878,2.250316,367.316834,"[-0.110296, 0.0504573, 0.00468473, -0.0105098,...",Oliver
13,809.2,152.0,438.6,516.0,1.524655,1.898137,790.153174,"[0.0554316, 0.0860607, -0.00389893, 0.0320879,...",Alberto
14,1125.4,104.0,316.2,374.0,1.490240,2.710115,600.989129,"[-0.0656685, 0.047579, -0.100778, 0.00661445, ...",Lucas
16,306.0,124.0,387.6,458.0,1.565820,2.250340,367.316834,"[-0.110296, 0.0504573, 0.00468473, -0.0105098,...",Oliver


### Extraction of embeddings information into different column to feed model

In [95]:
#Extract embedding informations and rearrange it into columns 
#(one column for each features - 128 columns in total)
em = [[] for i in range(len(XY.iloc[0].embedding))]

for index, r in XY.iterrows():
    for column in range (len(XY.iloc[0].embedding)):
         em[column].append(r.embedding[column])

In [96]:
#data = np.asarray(em).T

In [97]:
#Insert nex feature's columns into dataframe
for column in range (len(XY.iloc[0].embedding)):
    XY[column] = em[column]

In [98]:
# Drop unuseful column
XY.drop('embedding', 1, inplace=True)

In [99]:
XY

,trackX,trackY,trackW,trackH,height,distance,blur,GroundTrue,0,1,...,118,119,120,121,122,123,124,125,126,127
0,326.4,124.0,387.6,458.0,1.571027,2.248792,367.316834,Oliver,-0.110296,0.050457,...,-0.001748,-0.086560,0.028729,0.066900,-0.051501,-0.029163,0.052923,-0.105883,0.100579,-0.002058
1,802.4,150.0,442.0,520.0,1.519628,1.882822,790.153174,Alberto,0.055432,0.086061,...,-0.033809,-0.125701,0.021227,0.139772,-0.022684,-0.037909,0.186199,-0.067317,0.077795,0.044126
2,1135.6,108.0,309.4,364.0,1.475796,2.760290,600.989129,Lucas,-0.065669,0.047579,...,-0.025146,-0.081062,0.052214,0.186648,-0.156097,0.029255,0.069186,-0.105391,-0.135352,-0.108049
4,323.0,124.0,387.6,456.0,1.569231,2.253340,367.316834,Oliver,-0.110296,0.050457,...,-0.001748,-0.086560,0.028729,0.066900,-0.051501,-0.029163,0.052923,-0.105883,0.100579,-0.002058
5,805.8,152.0,438.6,518.0,1.519507,1.894786,790.153174,Alberto,0.055432,0.086061,...,-0.033809,-0.125701,0.021227,0.139772,-0.022684,-0.037909,0.186199,-0.067317,0.077795,0.044126
6,1128.8,104.0,312.8,370.0,1.486020,2.723061,600.989129,Lucas,-0.065669,0.047579,...,-0.025146,-0.081062,0.052214,0.186648,-0.156097,0.029255,0.069186,-0.105391,-0.135352,-0.108049
12,306.0,124.0,387.6,458.0,1.567878,2.250316,367.316834,Oliver,-0.110296,0.050457,...,-0.001748,-0.086560,0.028729,0.066900,-0.051501,-0.029163,0.052923,-0.105883,0.100579,-0.002058
13,809.2,152.0,438.6,516.0,1.524655,1.898137,790.153174,Alberto,0.055432,0.086061,...,-0.033809,-0.125701,0.021227,0.139772,-0.022684,-0.037909,0.186199,-0.067317,0.077795,0.044126
14,1125.4,104.0,316.2,374.0,1.490240,2.710115,600.989129,Lucas,-0.065669,0.047579,...,-0.025146,-0.081062,0.052214,0.186648,-0.156097,0.029255,0.069186,-0.105391,-0.135352,-0.108049
16,306.0,124.0,387.6,458.0,1.565820,2.250340,367.316834,Oliver,-0.110296,0.050457,...,-0.001748,-0.086560,0.028729,0.066900,-0.051501,-0.029163,0.052923,-0.105883,0.100579,-0.002058


In [100]:
print XY.columns

Index([    u'trackX',     u'trackY',     u'trackW',     u'trackH',
           u'height',   u'distance',       u'blur', u'GroundTrue',
                   0,             1,
       ...
                 118,           119,           120,           121,
                 122,           123,           124,           125,
                 126,           127],
      dtype='object', length=136)


### Creation of dataset for label 1
Creation of dataframe with the same person information/features for two different detection.
Will be labeled as 1 (label for the same person)

In [101]:
# Group by name - then work on one group to create same person dataset
groupBy = XY.groupby('GroundTrue')

In [102]:
def same(group):
    listToConcatSame = []
    
    for index, r in group.reset_index().drop('index', 1).iterrows():
        # Print percentage since could be long:
        percentage = index * 100 / len(group)
        if percentage % 10 == 0 :
            print str(percentage) + "%"
        
        # NumpyArray/Dataframe with same information for same detection repeated enought
        data = np.asarray([r for i in range(len(group))])
        left = pd.DataFrame(data, columns=group.columns)
        
        # Create a copy of the group (return a dataframe of all the detection for the same person)
        copy = group.copy()
        # Set index to avoid non compatibility between the 2 dataframes
        copy.set_index(left.index, inplace=True)
        
        #concate the 2 dateframes with a suffix "_right" for the dataframe of all the detection for the same person
        for col in left.columns:
            copy[str(col) + '_right'] = left[col]
        
        # Concatenate dataframe to return all possible combination
        listToConcatSame.append(copy)
        
    return pd.concat(listToConcatSame, ignore_index=True)

In [103]:
# Same person dataset
listToConcat = []
oneLenght = 0
for name, group in groupBy:
    print name
    oneLenght = oneLenght + len(group) * len(group)
    listToConcat.append(same(group))

Alberto
0%
0%
0%
10%
10%
20%
20%
30%
30%
40%
40%
40%
50%
50%
50%
60%
60%
60%
70%
70%
70%
80%
80%
80%
90%
90%
90%
Lucas
0%
0%
0%
10%
10%
20%
20%
20%
30%
30%
30%
40%
40%
50%
50%
50%
60%
60%
60%
70%
70%
80%
80%
80%
90%
90%
90%
Oliver
0%
0%
0%
10%
10%
10%
20%
20%
20%
30%
30%
30%
40%
40%
40%
50%
50%
50%
60%
60%
60%
70%
70%
70%
80%
80%
80%
90%
90%
90%
Sabrine
0%
10%
20%
30%
40%
50%
60%
70%
80%
90%


### Creation of dataset for label 0
Creation of dataframe of pair of different person information/features.
Will be labeled as 0 (label for two different persons)

In [104]:
def different(group1, group2):
    listToConcatDiff = []
    
    for index, r in group1.reset_index().drop('index', 1).iterrows():  
        # Display percentage ince could be long
        percentage = index * 100 / len(group1)
        if percentage % 20 == 0 :
            print str(percentage) + "%"
        
        # NumpyArray/Dataframe with same information for same detection repeated enought
        data = np.asarray([r for i in range(len(group2))])
        left = pd.DataFrame(data, columns=group1.columns)
        
        # Create a copy of the other group (return a dataframe of all the detection for the other person)
        copy = group2.copy()
        # Set index to avoid non compatibility between the 2 dataframes
        #copy.set_index(left.index, inplace=True)
        
        #concate the 2 dateframes with a suffix "_right" for the dataframe of all the detection of the other person
        for col in left.columns:
            copy[str(col) + '_right'] = left[col].values
        
        # Concatenate dataframe to return all possible combination
        listToConcatDiff.append(copy)
        
        if copy.isnull().values.any():
            print 'copy nan'      
    return pd.concat(listToConcatDiff, ignore_index=True)

In [105]:
# different person dataset
zeroLenght = 0
for name1, group1 in groupBy:
    print name1
    for name2, group2 in groupBy:
        if not name1 == name2:
            zeroLenght = zeroLenght + (len(group1) * len(group2))
            print '--> ' + name2
            listToConcat.append(different(group1, group2))
            
#bigDiff = pd.concat(listToConcat, ignore_index=True)

Alberto
--> Lucas
0%
0%
0%
20%
20%
40%
40%
40%
60%
60%
60%
80%
80%
80%
--> Oliver
0%
0%
0%
20%
20%
40%
40%
40%
60%
60%
60%
80%
80%
80%
--> Sabrine
0%
0%
0%
20%
20%
40%
40%
40%
60%
60%
60%
80%
80%
80%
Lucas
--> Alberto
0%
0%
0%
20%
20%
20%
40%
40%
60%
60%
60%
80%
80%
80%
--> Oliver
0%
0%
0%
20%
20%
20%
40%
40%
60%
60%
60%
80%
80%
80%
--> Sabrine
0%
0%
0%
20%
20%
20%
40%
40%
60%
60%
60%
80%
80%
80%
Oliver
--> Alberto
0%
0%
0%
20%
20%
20%
40%
40%
40%
60%
60%
60%
80%
80%
80%
--> Lucas
0%
0%
0%
20%
20%
20%
40%
40%
40%
60%
60%
60%
80%
80%
80%
--> Sabrine
0%
0%
0%
20%
20%
20%
40%
40%
40%
60%
60%
60%
80%
80%
80%
Sabrine
--> Alberto
0%
20%
40%
60%
80%
--> Lucas
0%
20%
40%
60%
80%
--> Oliver
0%
20%
40%
60%
80%


In [106]:
# add the label - one as the pair are from the same person
#bigDiff['label'] = np.zeros

### Creation of the features matrix and label vector

In [107]:
big = pd.concat(listToConcat, ignore_index=True)

In [108]:
del listToConcat, XY

In [109]:
label = np.concatenate([np.zeros(zeroLenght), np.ones(oneLenght)])

In [110]:
big['label'] = label

In [111]:
#big.to_pickle('bigML')

In [2]:
big = np.load('bigML')

In [3]:
big.columns

Index([    u'trackX',     u'trackY',     u'trackW',     u'trackH',
           u'height',   u'distance',       u'blur', u'GroundTrue',
                   0,             1,
       ...
        u'119_right',  u'120_right',  u'121_right',  u'122_right',
        u'123_right',  u'124_right',  u'125_right',  u'126_right',
        u'127_right',      u'label'],
      dtype='object', length=273)

In [114]:
big.isnull().values.any()

False

In [3]:
X = big.drop(['GroundTrue', 'GroundTrue_right', 'label',
             'trackX', 'trackY', 'trackW', 'trackH', 'trackX_right',
              'trackY_right', 'trackW_right', 'trackH_right'], 1)

In [4]:
Y = big.label

In [5]:
del big

In [ ]:
X.drop(['trackX', 'trackY', 'trackW', 'trackH', 'trackX_right', 'trackY_right', 'trackW_right', 'trackH_right'],
       1, inplace=True)

In [8]:
Y.head()

0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: label, dtype: float64

In [6]:
seizeX0 = X[X.index % 16 == 0]
seizeY0 = Y[Y.index % 16 == 0]

pd.to_pickle(seizeX0, 'seizeX0')
pd.to_pickle(seizeY0, 'seizeY0')
del seizeX0, seizeY0

seizeX1 = X[X.index % 16 == 1]
seizeY1 = Y[Y.index % 16 == 1]

pd.to_pickle(seizeX1, 'seizeX1')
pd.to_pickle(seizeY1, 'seizeY1')
del seizeX1, seizeY1

seizeX2 = X[X.index % 16 == 2]
seizeY2 = Y[Y.index % 16 == 2]

pd.to_pickle(seizeX2, 'seizeX2')
pd.to_pickle(seizeY2, 'seizeY2')
del seizeX2, seizeY2

seizeX3 = X[X.index % 16 == 3]
seizeY3 = Y[Y.index % 16 == 3]

pd.to_pickle(seizeX3, 'seizeX3')
pd.to_pickle(seizeY3, 'seizeY3')
del seizeX3, seizeY3

seizeX4 = X[X.index % 16 == 4]
seizeY4 = Y[Y.index % 16 == 4]

pd.to_pickle(seizeX4, 'seizeX4')
pd.to_pickle(seizeY4, 'seizeY4')
del seizeX4, seizeY4

seizeX5 = X[X.index % 16 == 5]
seizeY5 = Y[Y.index % 16 == 5]

pd.to_pickle(seizeX5, 'seizeX5')
pd.to_pickle(seizeY5, 'seizeY5')
del seizeX5, seizeY5

seizeX6 = X[X.index % 16 == 6]
seizeY6 = Y[Y.index % 16 == 6]

pd.to_pickle(seizeX6, 'seizeX6')
pd.to_pickle(seizeY6, 'seizeY6')
del seizeX6, seizeY6

seizeX7 = X[X.index % 16 == 7]
seizeY7 = Y[Y.index % 16 == 7]

pd.to_pickle(seizeX7, 'seizeX7')
pd.to_pickle(seizeY7, 'seizeY7')
del seizeX7, seizeY7

seizeX8 = X[X.index % 16 == 8]
seizeY8 = Y[Y.index % 16 == 8]

pd.to_pickle(seizeX8, 'seizeX8')
pd.to_pickle(seizeY8, 'seizeY8')
del seizeX8, seizeY8

print 'middle'

seizeX9 = X[X.index % 16 == 9]
seizeY9 = Y[Y.index % 16 == 9]

pd.to_pickle(seizeX9, 'seizeX9')
pd.to_pickle(seizeY9, 'seizeY9')
del seizeX9, seizeY9

seizeX10 = X[X.index % 16 == 10]
seizeY10 = Y[Y.index % 16 == 10]

pd.to_pickle(seizeX10, 'seizeX10')
pd.to_pickle(seizeY10, 'seizeY10')
del seizeX10, seizeY10

seizeX11 = X[X.index % 16 == 11]
seizeY11 = Y[Y.index % 16 == 11]

pd.to_pickle(seizeX11, 'seizeX11')
pd.to_pickle(seizeY11, 'seizeY11')
del seizeX11, seizeY11

seizeX12 = X[X.index % 16 == 12]
seizeY12 = Y[Y.index % 16 == 12]

pd.to_pickle(seizeX12, 'seizeX12')
pd.to_pickle(seizeY12, 'seizeY12')
del seizeX12, seizeY12

seizeX13 = X[X.index % 16 == 13]
seizeY13 = Y[Y.index % 16 == 13]

pd.to_pickle(seizeX13, 'seizeX13')
pd.to_pickle(seizeY13, 'seizeY13')
del seizeX13, seizeY13

seizeX14 = X[X.index % 16 == 14]
seizeY14 = Y[Y.index % 16 == 14]

pd.to_pickle(seizeX14, 'seizeX14')
pd.to_pickle(seizeY14, 'seizeY14')
del seizeX14, seizeY14

print 'almost'

seizeX15 = X[X.index % 16 == 15]
seizeY15 = Y[Y.index % 16 == 15]

pd.to_pickle(seizeX15, 'seizeX15')
pd.to_pickle(seizeY15, 'seizeY15')
del seizeX15, seizeY15

print 'done'

middle
almost
done


In [7]:
del X, Y

In [46]:
seizeX.drop(['trackX', 'trackY', 'trackW', 'trackH'], 1, inplace=True)
seizeX.drop(['trackX_right'], 1, inplace=True)
seizeX.drop('trackY_right', 1, inplace=True)
seizeX.drop('trackW_right', 1, inplace=True)
seizeX.drop('trackH_right', 1, inplace=True)

ValueError: labels ['trackW_right'] not contained in axis

In [15]:
del X, Y

In [32]:
'''bestC = 100
bestMean = 0

for i in range(83, 85, 1):
    print '**************************'
    oneVsAll = OneVsRestClassifier(LinearSVC(C=i))
    cross = cross_val_score(oneVsAll, X, Y, cv=10, verbose=10)
    mean = cross.mean()
    if bestMean < mean:
        bestC = i
        bestMean = mean''''''

**************************
[CV]  ................................................................
[CV] ................................. , score=0.500000, total=   1.0s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.0s remaining:    0.0s


[CV] ................................. , score=0.230769, total=   1.1s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    2.1s remaining:    0.0s


[CV] ................................. , score=0.296703, total=   1.1s
[CV]  ................................................................


...

[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    8.8s remaining:    0.0s


[CV] ................................. , score=0.288889, total=   1.0s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    9.7s remaining:    0.0s


[CV] ................................. , score=0.460674, total=   1.0s


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:   10.8s finished


In [24]:
clf = MLPClassifier(solver='sgd', alpha=1e-5,
                     hidden_layer_sizes=(6, 2), random_state=1)

In [31]:
besta = 0
bestScore = 0
for alpha in range(0, 10, 1):
    a = float(alpha)/10
    print 'a = ' + str(a)
    clf = MLPClassifier(solver='sgd', alpha=a,
                         hidden_layer_sizes=(6, 2), random_state=1)
    all_classes = np.array([0, 1])
    for i in range(15): # as we will use the last one for testing 
        X = np.load('seizeX' + str(i))
        Y = np.load('seizeY' + str(i))
        clf.partial_fit(X, Y, classes=all_classes)
        print i

    X = np.load('seizeX15')
    Y = np.load('seizeY15')
    score = clf.score(X, Y)
    if bestScore < score:
        bestScore = score
        besta = a

a = 0
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
a = 0
0
1
2


KeyboardInterrupt: 

In [30]:
print besta

0


In [26]:
X = np.load('seizeX15')
Y = np.load('seizeY15')
clf.score(X, Y)

0.7234464219139376

In [11]:
help(clf.partial_fit())

AttributeError: partial_fit is only available for stochastic optimizer. lbfgs is not stochastic

In [19]:
X = np.load('seizeX0')
Y = np.load('seizeY0')
cross = cross_val_score(clf, X, Y, cv=4, verbose=100)

[CV]  ................................................................
[CV] ................................. , score=0.723434, total=   1.4s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.5s remaining:    0.0s
[CV]  ................................................................
[CV] ................................. , score=0.723434, total=   1.3s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    2.9s remaining:    0.0s
[CV]  ................................................................
[CV] ................................. , score=0.723469, total=   1.4s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    4.3s remaining:    0.0s
[CV]  ................................................................
[CV] ................................. , score=0.723469, total=   1.3s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    5.7s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    5.8s finished


In [20]:
cross.mean()

0.72345184974005294

In [34]:
bestMean

0.41279513245805377

In [17]:
cross = cross_val_score(oneVsAll, X, Y, cv=10, verbose=100)

[CV]  ................................................................
[CV] ................................. , score=0.293478, total=   1.4s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.4s remaining:    0.0s
[CV]  ................................................................
[CV] ................................. , score=0.285714, total=   1.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    2.4s remaining:    0.0s
[CV]  ................................................................
[CV] ................................. , score=0.494505, total=   1.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    3.5s remaining:    0.0s
[CV]  ................................................................
[CV] ................................. , score=0.422222, total=   1.1s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    4.5s remaining:    0.0s
[CV]  ................................................................
[CV] ................................. , 

In [18]:
cross.mean()

0.38915507252341647

In [ ]:
#X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=0)

In [ ]:
#polynomial_svm.fit(X_train, y_train)

In [ ]:
#polynomial_svm.score(X_test, y_test)